# KNN (K-Nearest-Neighbors)

KNN is a simple concept: define some distance metric between the items in your dataset, and find the K closest items. You can then use those items to predict some property of a test item, by having them somehow "vote" on it.

As an example, let's look at the MovieLens data. We'll try to guess the rating of a movie by looking at the 10 movies that are closest to it in terms of genres and popularity.

To start, we'll load up every rating in the data set into a Pandas DataFrame:

In [1]:
import pandas as pd

r_cols = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv('C:\\Users\\kevin\\Desktop\\Python Lessons\\DataScience-Python3\\ml-100k\\u.data', sep='\t', names=r_cols, usecols=range(3))
ratings.head()


,user_id,movie_id,rating
0,0,50,5
1,0,172,5
2,0,133,1
3,196,242,3
4,186,302,3


Now, we'll group everything by movie ID, and compute the total number of ratings (each movie's popularity) and the average rating for every movie:

In [2]:
import numpy as np

movieProperties = ratings.groupby('movie_id').agg({'rating': [np.size, np.mean]})
#use aggregate function to get size and mean rating. 
movieProperties.head()

rating          
           size      mean
movie_id                 
1           452  3.878319
2           131  3.206107
3            90  3.033333
4           209  3.550239
5            86  3.302326

The raw number of ratings isn't very useful for computing distances between movies, so we'll create a new DataFrame that contains the normalized number of ratings. So, a value of 0 means nobody rated it, and a value of 1 will mean it's the most popular movie there is.

In [3]:
movieNumRatings = pd.DataFrame(movieProperties['rating']['size'])
movieNormalizedNumRatings = movieNumRatings.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
#standardize the size by dividing the sizes by the range
#of the dataset. 
movieNormalizedNumRatings.head()

,size
movie_id,
1,0.773585
2,0.222985
3,0.152659
4,0.356775
5,0.145798


Now, let's get the genre information from the u.item file. The way this works is there are 19 fields, each corresponding to a specific genre - a value of '0' means it is not in that genre, and '1' means it is in that genre. A movie may have more than one genre associated with it.

While we're at it, we'll put together everything into one big Python dictionary called movieDict. Each entry will contain the movie name, list of genre values, the normalized popularity score, and the average rating for each movie:

In [4]:
movieDict = {}
#empty dictionary. 
with open(r'C:\\Users\\kevin\\Desktop\\Python Lessons\\DataScience-Python3\\ml-100k\\u.item') as f:
    temp = ''
    for line in f:
        #line.decode("ISO-8859-1")
        fields = line.rstrip('\n').split('|')
        movieID = int(fields[0])
        z = fields[1]
        genres = fields[5:25]
        genres = map(int, genres)
        movieDict[movieID] = (name, np.array(list(genres)), movieNormalizedNumRatings.loc[movieID].get('size'), movieProperties.loc[movieID].rating.get('mean'))

#extract movie name, genres and ID

For example, here's the record we end up with for movie ID 1, "Toy Story":

In [5]:
print(movieDict[1])

#genre indicator, popularity score and average rating. 

('Toy Story (1995)', array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 0.77358490566037741, 3.8783185840707963)


Now let's define a function that computes the "distance" between two movies based on how similar their genres are, and how similar their popularity is. Just to make sure it works, we'll compute the distance between movie ID's 2 and 4:

In [26]:
from scipy import spatial

def ComputeDistance(a, b):
    genresA = a[1]
    genresB = b[1]
    genreDistance = spatial.distance.cosine(genresA, genresB)
    #use a cosine similarity metric. 
    popularityA = a[2]
    popularityB = b[2]
    popularityDistance = abs(popularityA - popularityB)
    #absolute difference between popularity score. 
    avgratingA = a[3]
    avgratingB = b[3]
    avgratingDist = abs(avgratingA - avgratingB)
    #I ADDED THE AVG RATING PARAMETER AS PART OF THE EXERCISE
    return genreDistance + popularityDistance + avgratingDist
    
ComputeDistance(movieDict[393], movieDict[359])

#these movies are quite different because they had
#0.8 distance scores. 



1.3869979273870783

Remember the higher the distance, the less similar the movies are. Let's check what movies 2 and 4 actually are - and confirm they're not really all that similar:

In [27]:
print(movieDict[393])
print(movieDict[259])


('Mrs. Doubtfire (1993)', array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 0.32761578044596912, 3.4114583333333335)
('George of the Jungle (1997)', array([0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 0.27615780445969124, 2.6851851851851851)


Now, we just need a little code to compute the distance between some given test movie (Toy Story, in this example) and all of the movies in our data set. When the sort those by distance, and print out the K nearest neighbors:

In [28]:
import operator

def getNeighbors(movieID, K):
    distances = []
    for movie in movieDict:
        if (movie != movieID):
            dist = ComputeDistance(movieDict[movieID], movieDict[movie])
            #iterate through every movie and find the distance score. 
            distances.append((movie, dist))
            #add distance score to the list. 
    distances.sort(key=operator.itemgetter(1)) #sort by distance score
    neighbors = []
    for x in range(K):
        neighbors.append(distances[x][0])
    return neighbors

K = 10
avgRating = 0
neighbors = getNeighbors(393, K)
for neighbor in neighbors:
    avgRating += movieDict[neighbor][3]
    print (movieDict[neighbor][0] + " " + str(movieDict[neighbor][3]))
    
avgRating /= K
#we get a bunch of children and comedy movies. Not a lot of Disney though. 

Mighty Aphrodite (1995) 3.41847826087
That Thing You Do! (1996) 3.46590909091
Clueless (1995) 3.45882352941
Happy Gilmore (1996) 3.38926174497
In & Out (1997) 3.30434782609
Birdcage, The (1996) 3.44368600683
Brothers McMullen, The (1995) 3.45205479452
Mallrats (1995) 3.44444444444
Austin Powers: International Man of Mystery (1997) 3.24615384615
Dazed and Confused (1993) 3.46875


While we were at it, we computed the average rating of the 10 nearest neighbors to Toy Story:

In [29]:
avgRating

3.4091909544188597

How does this compare to Toy Story's actual average rating?

In [31]:
movieDict[393]

('Mrs. Doubtfire (1993)',
 array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 0.32761578044596912,
 3.4114583333333335)

Not too bad!


## Activity

Our choice of 10 for K was arbitrary - what effect do different K values have on the results?

Our distance metric was also somewhat arbitrary - we just took the cosine distance between the genres and added it to the difference between the normalized popularity scores. Can you improve on that?

What are other parameters I can use to find the distance between movies? 
- producer, like Disney?
- year of release?
- average rating? I tried this one. 
